In [1]:
using IntervalArithmetic
using LaTeXStrings
using Plots

In [2]:
include("./heatmap/index.jl")
include("./method1/index.jl")
include("./method2/index.jl")
include("./method3/index.jl")
include("./method4/index.jl")
include("./intervalPCM/index.jl")
include("./twofoldInterval/index.jl")
include("./twofoldIntervalPCM/index.jl")

print()

In [3]:
function method1(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    A = intervalPCM2TwofoldIntervalPCM(A₁, A₂)
    result = solveLP_m1(A)
    Â = updatePCM_m1(A, result)
    return Â
end

method1 (generic function with 1 method)

In [4]:
function  method2(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    εᶿ = solveIndividualCancellingLP_m2([A₁, A₂])
    result = solveApproximationLP_m2([A₁, A₂], εᶿ)
    A = importance2TwofoldIntervalPCM_m2(result)
    return A
end

method2 (generic function with 1 method)

In [5]:
function method3(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    approximationResults = solveApproximationLP_m3.([A₁, A₂])
    tBoundaries = calculateTBoundaries_m3.(approximationResults)
    result = solveConcatLP_m3(approximationResults, tBoundaries)
    Â = generatePCM_m3(result)
    return Â
end

method3 (generic function with 1 method)

In [6]:
function method4(A₁::Matrix{Interval{T}}, A₂::Matrix{Interval{T}}) where {T <: Real}
    results = solveApproximationLP_m4.([A₁, A₂])
    A = generateTwofoldIntervalMatrix_m4(results)
    result = solveCancellingLP_m4(A)
    Â = updatePCM_m4(A, result)
    return Â
end

method4 (generic function with 1 method)

In [7]:
function displayAll(
        A₁::Matrix{Interval{T}},
        A₂::Matrix{Interval{T}}) where {T <: Real}
    Â₁ = method1(A₁, A₂)
    Â₂ = method2(A₁, A₂)
    Â₃ = method3(A₁, A₂)
    Â₄ = method4(A₁, A₂)

    println("\\mathscr{A}^1 = $(twofoldIntervalMatrixLaTeXString(Â₁))"); println()
    println("\\mathscr{A}^2 = $(twofoldIntervalMatrixLaTeXString(Â₂))"); println()
    println("\\mathscr{A}^3 = $(twofoldIntervalMatrixLaTeXString(Â₃))"); println()
    println("\\mathscr{A}^4 = $(twofoldIntervalMatrixLaTeXString(Â₄))"); println()
end

displayAll (generic function with 1 method)

# example 1

In [9]:
A₁ = [
    1..1 (1/2)..2 (1/2)..2
    (1/2)..2 1..1 (1/2)..2
    (1/2)..2 (1/2)..2 1..1
]
_, n = size(A₁)
A₂ = fill(1..1, (n, n))
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 4 / 5)..(A₁[i,j].hi * 5 / 4)
end

println("A_1 = A_2 = $(intervalMatrixLaTeXString(A₁))")
println()

displayAll(A₁, A₂)

A_1 = A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 0.5, 2.0 \right] & \left[ 0.5, 2.0 \right] \\ \left[ 0.5, 2.0 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.5, 2.0 \right] \\ \left[ 0.5, 2.0 \right] & \left[ 0.5, 2.0 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}

\mathscr{A}^1 = \begin{bmatrix} \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[   0.4, \left[   0.5,   2.0 \right],   2.5 \right] & \left[   0.4, \left[   0.5,   2.0 \right],   2.5 \right] \\ \left[   0.4, \left[   0.5,   2.0 \right],   2.5 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[   0.4, \left[   0.5,   2.0 \right],   2.5 \right] \\ \left[   0.4, \left[   0.5,   2.0 \right],   2.5 \right] & \left[   0.4, \left[   0.5,   2.0 \right],   2.5 \right] & \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] \end{bmatrix}

\mathscr{A}^2 = \begin{bmatrix} \left[   1.0, \left[   1.0,   1.0 \right],   1.0 \right] & \left[   0.4, \left[   0.5,   2.0 \right],   2.5 \righ

# example 2

In [10]:
A₁ = [
    1..1 1..2 (2/3)..(4/3) 1..(4/3)
    (1/2)..1 1..1 (4/9)..1 (2/3)..1
    (3/4)..(3/2) 1..(9/4) 1..1 1..(3/2)
    (3/4)..1 1..(3/2) (2/3)..1 1..1
]
_, n = size(A₁)
A₂ = fill(1..1, (n, n))
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 4 / 5)..(A₁[i,j].hi * 5 / 4)
end

println("A_1 = $(intervalMatrixLaTeXString(A₁))")
println()
println("A_2 = $(intervalMatrixLaTeXString(A₂))")
println()

displayAll(A₁, A₂)

A_1 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.0, 2.0 \right] & \left[ 0.667, 1.333 \right] & \left[ 1.0, 1.333 \right] \\ \left[ 0.5, 1.0 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.444, 1.0 \right] & \left[ 0.667, 1.0 \right] \\ \left[ 0.75, 1.5 \right] & \left[ 1.0, 2.25 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.0, 1.5 \right] \\ \left[ 0.75, 1.0 \right] & \left[ 1.0, 1.5 \right] & \left[ 0.667, 1.0 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}

A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 0.8, 2.5 \right] & \left[ 0.533, 1.667 \right] & \left[ 0.8, 1.667 \right] \\ \left[ 0.4, 1.25 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.356, 1.25 \right] & \left[ 0.533, 1.25 \right] \\ \left[ 0.6, 1.875 \right] & \left[ 0.8, 2.812 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.8, 1.875 \right] \\ \left[ 0.6, 1.25 \right] & \left[ 0.8, 1.875 \right] & \left[ 0.533, 1.25 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}

\mathscr{A}^1 = \begin{bmatrix} \left[   1.0, \left[

<!-- ともに弱い整合性を持ち， $A_{1ij} \subseteq A_{2ij}$

```
A₁ = [
    1..1 2..5 2..4 2..3
    (1/5)..(1/2) 1..1 (1/5)..4 (1/5)..3
    (1/4)..(1/2) (1/4)..5 1..1 (1/4)..3
    (1/3)..(1/2) (1/3)..5 (1/3)..4 1..1
]
A₂ = fill(1..1, size(A₁))
_, n = size(A₁)
for i = 1:n, j = 1:n
    if i == j continue end
    A₂[i,j] = (A₁[i,j].lo * 1/1.1)..(A₁[i,j].hi * 1.1)
end

display(L"A_1 = %$(intervalMatrixLaTeXString(A₁))")
display(L"A_2 = %$(intervalMatrixLaTeXString(A₂))")

display(isConsistentIntervalPCM(A₁))
display(isWeaklyConsistentIntervalPCM(A₁))
display(isConsistentIntervalPCM(A₂))
display(isWeaklyConsistentIntervalPCM(A₂))

plotAll(A₁, A₂)
``` -->

# example 3

In [11]:
A = [
    1 2 4 4
    1/2 1 2 2
    1/4 1/2 1 1
    1/4 1/2 1 1
]

A₁ = randamizedIntervalPCM(A, 1234, 3e-2)
A₂ = randamizedIntervalPCM(A, 134, 3e-2)

println("A_1 = $(intervalMatrixLaTeXString(A₁))")
println()
println("A_2 = $(intervalMatrixLaTeXString(A₂))")
println()

displayAll(A₁, A₂)

A_1 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.981, 2.033 \right] & \left[ 3.974, 4.109 \right] & \left[ 3.958, 4.048 \right] \\ \left[ 0.492, 0.505 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.944, 2.048 \right] & \left[ 1.971, 2.045 \right] \\ \left[ 0.243, 0.252 \right] & \left[ 0.488, 0.515 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.983, 1.022 \right] \\ \left[ 0.247, 0.253 \right] & \left[ 0.489, 0.507 \right] & \left[ 0.978, 1.017 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}

A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.998, 2.01 \right] & \left[ 3.974, 4.05 \right] & \left[ 3.962, 4.041 \right] \\ \left[ 0.497, 0.5 \right] & \left[ 1.0, 1.0 \right] & \left[ 1.945, 2.015 \right] & \left[ 1.942, 2.033 \right] \\ \left[ 0.247, 0.252 \right] & \left[ 0.496, 0.514 \right] & \left[ 1.0, 1.0 \right] & \left[ 0.972, 1.021 \right] \\ \left[ 0.247, 0.252 \right] & \left[ 0.492, 0.515 \right] & \left[ 0.979, 1.029 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}



# example 4

In [13]:
A = [
    1 2 3 10
    1/2 1 3 8
    1/3 1/3 1 3
    1/10 1/8 1/3 1
]

A₁ = randamizedIntervalPCM(A, 124, 4e-1)
A₂ = randamizedIntervalPCM(A, 411, 4e-1)

println("A_1 = $(intervalMatrixLaTeXString(A₁))")
println()
println("A_2 = $(intervalMatrixLaTeXString(A₂))")
println()

displayAll(A₁, A₂)

A_1 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.415, 2.501 \right] & \left[ 2.252, 4.04 \right] & \left[ 8.141, 13.465 \right] \\ \left[ 0.4, 0.707 \right] & \left[ 1.0, 1.0 \right] & \left[ 2.122, 3.056 \right] & \left[ 6.481, 8.402 \right] \\ \left[ 0.248, 0.444 \right] & \left[ 0.327, 0.471 \right] & \left[ 1.0, 1.0 \right] & \left[ 2.174, 3.356 \right] \\ \left[ 0.074, 0.123 \right] & \left[ 0.119, 0.154 \right] & \left[ 0.298, 0.46 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}

A_2 = \begin{bmatrix} \left[ 1.0, 1.0 \right] & \left[ 1.547, 2.232 \right] & \left[ 2.233, 3.723 \right] & \left[ 8.746, 12.652 \right] \\ \left[ 0.448, 0.646 \right] & \left[ 1.0, 1.0 \right] & \left[ 2.28, 3.644 \right] & \left[ 5.559, 10.701 \right] \\ \left[ 0.269, 0.448 \right] & \left[ 0.274, 0.439 \right] & \left[ 1.0, 1.0 \right] & \left[ 2.759, 3.774 \right] \\ \left[ 0.079, 0.114 \right] & \left[ 0.093, 0.18 \right] & \left[ 0.265, 0.362 \right] & \left[ 1.0, 1.0 \right] \end{bmatrix}
